In [1]:
# from src.utils.setCurrDirToBase import setCurrDirToBase
# setCurrDirToBase()

In [2]:
import json
# FOLDER CHECK

from ta_assignment_automation import create_required_folders
create_required_folders()

# Scripts
from ta_assignment_automation import parseCourses
from ta_assignment_automation import parseSections
from ta_assignment_automation import parseSchedule
from ta_assignment_automation import combine_course_schedules
from ta_assignment_automation import parseTAData
from ta_assignment_automation import parseInstructorPref
from ta_assignment_automation import computeConflictMatrix
from ta_assignment_automation import create_final_sections
from ta_assignment_automation import duplicateTAs
from ta_assignment_automation import incremental_TA_duplication


In [3]:
# Utility files
from ta_assignment_automation import getConflictMatFromTxt
from ta_assignment_automation import showAssignments
from ta_assignment_automation import splitTAhours
from ta_assignment_automation import convertInputFilesToJSON


In [4]:
# Algorithms
from ta_assignment_automation import hungarian

# PHASE 1
### In this phase all the input files will be processed and the scripts will create courses, section, schedule, TA and instructor preference data.

In [5]:
convertInputFilesToJSON() # convert input CSV files to JSON for processing
parseCourses()
parseSections()
parseSchedule()
combine_course_schedules()
parseTAData()
parseInstructorPref()
create_final_sections()

Reading Course Data
Writing Course Data to JSON file:  output_files/courses.json
Total Courses added:  80
Reading Course Data to Add Sections
Writing Section Data to JSON file:  output_files/sections.json
Total Sections added:  579
Reading Course Data to Add Schedule
Writing Schedule Data to JSON file:  output_files/schedule.json
Total Schedule added:  715
Writing FINAL Data to JSON file:  output_files/combined_section_data.json
Total sections added in final data:  220
Writing FINAL Data to JSON file:  output_files/all_section_data.json
Total sections added in final data:  235
Reading TA Data
Writing TA Data to JSON file:  output_files/ta_schedule.json
Total TAs added:  53
Writing Ins Pref Data to JSON file:  output_files/ins_pref.json
Total preferences added:  42
READING COMBINED SECTION DATA
Processing Lab Data
{'section_id': 119, 'crn': [40417, 57780], 'course_no': 'CSCI1302', 'credit_hours': 4, 'course_long_name': 'Software Development', 'ins_fname': 'Bradley', 'ins_lname': 'Barnes

# Phase 2
### After all the data has been processed we proceed with creating the conflict matrix and running the hungarian algorithm

In [6]:
computeConflictMatrix()                                         # create conflict matrix and save it locally
conf_matrix = getConflictMatFromTxt()                           # get the matrix from the text file
assignments = hungarian(conf_matrix, print_results=False)       # Run the hungarian algorithm
print(assignments)                                      
showAssignments(assignments, conf_matrix)                       # Show assignments - will also create the final assignment data locally

Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt
Conflict matix loaded to use in algo
([13, 43, 52, 1, 2, 34, 9, 51, 5, 32, 3, 42, 7, 8, 6, 11, 26, 10, 12, 29, 30, 37, 14, 38, 33, 15, 39, 40, 41, 18, 31, 44, 45, 16, 17, 46, 4, 47, 48, 35, 50, 19, 49, 36, -1, -1, 20, 21, 22, -1, -1, -1, 23, -1, 24, -1, -1, 25, -1, 0, -1, 27, 28, -1, -1, 52], array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  4.,  8., 12., 16., 20., 24., 28., 32., 36., 40.,
       44., 48., 52., 56., 60., 64., 68., 72., 76., 80., 84., 88., 92.,
       96.]))
job 13 assigned to worker 0 with cost (j = 13, w = 0) = 4
job 43 assign

# Phase 3
### After the initial assignment, we need to assign the TAs who have been assigned for classes with less hours

In [7]:
# WE NEED TO DUPLICATE TAs WHO HAVE BEEN ASSIGNED TO CLASS LESS THAN 40 (CAPACITY_CAP) ENROLLMENT
duplicateTAs()
# RUN THE HUNGARIAN ON THE NEW CONFLICT MATRIX AGAIN
computeConflictMatrix()
conf_matrix = getConflictMatFromTxt()
assignments = hungarian(conf_matrix, print_results=False)
print(assignments)
showAssignments(assignments, conf_matrix, showLogs=False)

TOTAL TAs before Duplication:  53
TOTAL TAs after Duplication: 
63

Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt
Conflict matix loaded to use in algo
([53, 43, 55, 1, 2, 34, 9, 60, 5, 32, 3, 42, 7, 8, 6, 52, 26, 51, 11, 29, 59, 37, 14, 38, 33, 15, 39, 40, 41, 18, 31, 44, 45, 16, 17, 46, 4, 47, 48, 35, 50, 19, 49, 36, 10, 12, 20, 21, 22, 54, 56, 57, 23, 58, 24, 30, 13, 25, 61, 0, 62, 27, 28, -1, -1, 62], array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,
        20.,  24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,
        64., 

# Phase 4
### If there are any unassigned courses left -> we find the next available TA and run hungarian again
### This will happen untill all unassigned courses have been assigned or if the number of TAs become equal to the number of courses

In [8]:
# Check if courses are left unassigned
with open("assignment_output_files/unassigned_courses.json", 'r') as file:
    unassignedCourses = json.load(file)

# keep finding next available TA and run assignments until all unassigned courses have been assigned 
# or if the number of TAs become equal to the number of courses
while len(unassignedCourses) != 0:
    incremental_TA_duplication()
    computeConflictMatrix()
    conf_matrix = getConflictMatFromTxt()
    assignments = hungarian(conf_matrix, print_results=False)
    print(assignments)
    showAssignments(assignments, conf_matrix, showLogs=False)
    with open("assignment_output_files/unassigned_courses.json", 'r') as file:
        unassignedCourses = json.load(file)


showAssignments(assignments, conf_matrix)
splitTAhours() # SPLIT TA hours for TAs who have been assigned to more than one class

INCREMENTAL DUPLICATION: 
NO OF TAs: 64
Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt
Conflict matix loaded to use in algo
([53, 43, 55, 1, 2, 34, 9, 60, 5, 32, 3, 42, 7, 8, 6, 52, 26, 51, 11, 29, 59, 37, 14, 38, 33, 15, 39, 40, 41, 18, 31, 44, 45, 16, 17, 46, 4, 47, 48, 35, 50, 19, 49, 36, 10, 12, 20, 21, 22, 54, 56, 57, 23, 58, 24, 30, 13, 25, 61, 0, 62, 27, 28, 63, -1, 63], array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,
        20.,  24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,
        64.,  68.,  72.,  76.,  80.,  84.